<a href="https://colab.research.google.com/github/robinloh/H6751-Text-and-Web-Mining/blob/master/H6751_Text_and_Web_Mining_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!git clone https://github.com/ohguowei/H6751TextandWebMining2.git

fatal: destination path 'H6751TextandWebMining2' already exists and is not an empty directory.


In [0]:
!pip install contractions
!pip install textsearch
from nltk.corpus import stopwords
import contractions
import pandas as pd
import re as regex
pd.set_option('display.max_colwidth',5000)

In [74]:
url = "H6751TextandWebMining2/train.csv"
df = pd.read_csv(url) #READING
df.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0
1,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0


In [0]:
class Initialize():
    data = []
    processed_data = []
    wordlist = []

    data_model = None
    data_labels = None
    is_testing = False

    def initialize(self, csv_file):  # read wiki data
        
        self.data = pd.read_csv(csv_file, header=0, names=["id", "text", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])
        self.data = self.data[self.data["toxic"].isin(["0", "1"])]
        self.data = self.data[self.data["severe_toxic"].isin(["0", "1"])]
        self.data = self.data[self.data["obscene"].isin(["0", "1"])]
        self.data = self.data[self.data["threat"].isin(["0", "1"])]
        self.data = self.data[self.data["insult"].isin(["0", "1"])]
        self.data = self.data[self.data["identity_hate"].isin(["0", "1"])]
        
        self.processed_data = self.data
        self.wordlist = []
        self.data_model = None
        self.data_labels = None
    
class PreProcess:
    def iterate(self):
        for cleanup_method in [self.remove_extra_quotes,
                               self.remove_special_chars,
                               self.remove_numbers,
                               self.remove_non_ascii]:
            yield cleanup_method

    @staticmethod
    def remove_by_regex(comments, regexp):
        comments.loc[:, "text"].replace(regexp, "  ", inplace=True)  
        return comments
    
    def remove_special_chars(self, comments):
        for remove in map(lambda r: regex.compile(regex.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$", "@", "%", "^", "*", "(", ")", "{", "}", "_",
                                                                     "[", "]", "|", "/", "\\", ">", "<", "-", "!", "?", ".", "~", "\+", "--", "---", "#", '\"', "\t", "\n"]):
            comments.loc[:, "text"].replace(remove, "  ", inplace = True)
        return comments

    def remove_numbers(self, comments):
        return PreProcess.remove_by_regex(comments, regex.compile(r"\s?[0-9]+\.?[0-9]*"))

    def remove_non_ascii(self, comments):
        return PreProcess.remove_by_regex(comments, regex.compile(r"[^\x00-\x7F]+"))

    def remove_extra_quotes(self, comments):
        return PreProcess.remove_by_regex(comments, regex.compile(r"''+"))

class Clean(Initialize):
    def __init__(self, previous):
        self.processed_data = previous.processed_data
        
    def cleanup(self, cleanuper):
        t = self.processed_data
        for cleanup_method in cleanuper.iterate():
            t = cleanup_method(t)

        self.processed_data = t

In [0]:
data = Initialize()
data.initialize("H6751TextandWebMining2/train.csv")
# data.processed_data.head(50)

In [93]:
data = Clean(data)
data.cleanup(PreProcess())
data.processed_data.head()

,id,text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They weren't vandalisms just closure on some GAs after I voted at New York Dolls FAC And please don't remove the template from the talk page since I'm retired now,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww He matches this background colour I'm seemingly stuck with Thanks talk January UTC,0,0,0,0,0,0
2,000113f07ec002fd,Hey man I'm really not trying to edit war It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page He seems to care more about the formatting than the actual info,0,0,0,0,0,0
3,0001b41b1c6bb37e,More I can't make any real suggestions on improvement I wondered if the section statistics should be later on or a subsection of types of accidents I think the references may need tidying so that they are all in the exact same format ie date format etc I can do that later on if no one else does first if you have any preferences for formatting style on references or want to do it yourself please let me know There appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up It's listed in the relevant form eg Wikipedia Good article nominations Transport,0,0,0,0,0,0
4,0001d958c54c6e35,You sir are my hero Any chance you remember what page that's on,0,0,0,0,0,0


In [94]:
data.processed_data.text = data.processed_data.apply(lambda x: contractions.fix(x.text), axis = 1)
data.processed_data.head()

,id,text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They were not vandalisms just closure on some GAs after I voted at New York Dolls FAC And please do not remove the template from the talk page since I am retired now,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww He matches this background colour I am seemingly stuck with Thanks talk January UTC,0,0,0,0,0,0
2,000113f07ec002fd,Hey man I am really not trying to edit war it is just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page He seems to care more about the formatting than the actual info,0,0,0,0,0,0
3,0001b41b1c6bb37e,More I can not make any real suggestions on improvement I wondered if the section statistics should be later on or a subsection of types of accidents I think the references may need tidying so that they are all in the exact same format ie date format etc I can do that later on if no one else does first if you have any preferences for formatting style on references or want to do it yourself please let me know There appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up it is listed in the relevant form eg Wikipedia Good article nominations Transport,0,0,0,0,0,0
4,0001d958c54c6e35,You sir are my hero Any chance you remember what page that is on,0,0,0,0,0,0


In [95]:
data.processed_data.text = data.processed_data.apply(lambda x: regex.sub("[']", " ", x.text), axis = 1)
data.processed_data.text

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [96]:
data.processed_data.text = data.processed_data.apply(lambda x: regex.sub(" +", " ", x.text), axis = 1)
data.processed_data.text.head()

0                                                                                                                                                                                                                                                                                                                                                                      Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They were not vandalisms just closure on some GAs after I voted at New York Dolls FAC And please do not remove the template from the talk page since I am retired now 
1                                                                                                                                                                                                                                                                                                                                                                                                   

In [97]:
data.processed_data.text = data.processed_data.apply(lambda x: regex.sub("\++", " ", x.text), axis = 1)
data.processed_data.text.head()

0                                                                                                                                                                                                                                                                                                                                                                      Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They were not vandalisms just closure on some GAs after I voted at New York Dolls FAC And please do not remove the template from the talk page since I am retired now 
1                                                                                                                                                                                                                                                                                                                                                                                                   

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       